In [2]:
import pandas as pd
import MySQLdb
import os
import time

In [156]:
db = MySQLdb.connect("47.100.14.137","hyps44413","Nanhua")
db.set_character_set('gbk')

In [3]:
db = MySQLdb.connect("localhost","root","123456")
db.set_character_set("utf8")

# 單獨品種日資料

In [ ]:
DATA_DIR = "../CL Daily Data WTI原油期货-所有资料"
DATA_DIR = "../all IOE"

#SQL_INSERT("Iron_original_data",DATA_DIR)
#SQL_INSERT("WTI_Oil_original_data",DATA_DIR)

In [3]:
def SQL_INSERT(dbname,DATA_DIR):
    for filename in os.listdir(DATA_DIR):      
        cursor = db.cursor()
        sql = "CREATE DATABASE IF NOT EXISTS " + dbname
        cursor.execute(sql)

        contract = filename.split(".xlsx")[0]

        sql = "CREATE TABLE IF NOT EXISTS " dbname + "." + contract + \
              " (Date DATE PRIMARY KEY,\
                Open FLOAT(5),\
                High FLOAT(5),\
                Low FLOAT(5),\
                Close FLOAT(5),\
                Volume FLOAT(5),\
                OI FLOAT(5))"
        cursor.execute(sql)


        data = pd.read_excel(DATA_DIR+"/"+filename,header=None)
        data = data.fillna("NULL")


        for i in range(8,len(data)):
            cursor = db.cursor()
            sql = "insert into " + dbname + "." + contract + " values ('"+ data.iloc[i][0].strftime("%Y-%m-%d") + "',"\
                    + str(data.iloc[i][1]) + ","\
                    + str(data.iloc[i][2]) + ","\
                    + str(data.iloc[i][3]) + ","\
                    + str(data.iloc[i][4]) + ","\
                    + str(data.iloc[i][5]) + ","\
                    + str(data.iloc[i][6]) + ")"
            cursor.execute(sql)
            db.commit()
        cursor.close()

C:\Users\Philip\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: Warning: (1007, "Can't create database 'wti_oil_original_data'; database exists")
  


# 合約分開

In [153]:
DATA_DIR = '../ALL CONCRACT/鄭商所'
for filename in os.listdir(DATA_DIR):
    
    data = pd.read_excel(DATA_DIR + "/" + filename,header=None)
    
    folderName = filename.split(".xlsx")[0]
    
    name_column = 1
    column = 0
    next_column = 8
    while next_column < len(data.loc[0]):
        name = data.loc[0][name_column]
        split_data = data.iloc[8::,column:next_column]

        #刪除所有列都是空值的行
        split_data = split_data.dropna(how='all')

        #strip()刪除字串空白
        head=['Date','PX_OPEN','PX_HIGH','PX_LOW','PX_LAST','PX_SETTLE','VOLUME','OPEN_INT']

        split_data.to_csv('../ALL CONCRACT/CZCE/'+folderName+"/"+name.strip()+'.csv',header=head,index=None)
        column += 9
        next_column += 9
        name_column += 9

# 所有交易所日資料

In [157]:
FOLDER_DIR = '../ALL CONCRACT/CZCE'
for foldername in os.listdir(FOLDER_DIR):
    cursor = db.cursor()
    
    databaseName = foldername + "_day_data"
    
    sql = "create database if not exists " + databaseName
    cursor.execute(sql)
    
    DATA_DIR = FOLDER_DIR + "/" + foldername
    for filename in os.listdir(DATA_DIR):
        
        name = filename.split('.csv')[0]
        table_name = name.split(' ')[0]
        
        data = pd.read_csv(DATA_DIR + "/" + filename,header=0)
        data['Date'] = pd.to_datetime(data['Date'])

        data = data.where((pd.notnull(data)), None)
        
        sql = "create table if not exists "+ databaseName + "." + table_name + \
                  " (Date DATE PRIMARY KEY,\
                    Open FLOAT(5),\
                    High  FLOAT(5),\
                    Low  FLOAT(5),\
                    Close  FLOAT(5),\
                    Settle  FLOAT(5),\
                    Volume  FLOAT(5),\
                    OI  FLOAT(5))"
        cursor.execute(sql)
        
        if len(data) == 0:
            continue
        
        values = []
        for i in range(len(data)):
            values.append((data.iloc[i][0],data.iloc[i][1],data.iloc[i][2],data.iloc[i][3],data.iloc[i][4],
                           data.iloc[i][5],data.iloc[i][6],data.iloc[i][7]))
        sql = "insert into " + databaseName + "." + table_name + " values (%s,%s,%s,%s,%s,%s,%s,%s)"
        cursor.executemany(sql,values)
        db.commit()   
    cursor.close()

# 石油相關資料

In [127]:
DATA_DIR = '../DOC HO'

In [4]:
data = pd.read_excel("../DOC HO/CL.xlsx")

In [9]:
date = pd.read_excel("../20171120 所有换约日期.xlsx")

In [20]:
date['CL期货连续合约换约日期'] = pd.to_datetime(date['CL期货连续合约换约日期'])

In [183]:
test = test.sort_index(axis=0,ascending=False)
test = test.reset_index(drop=True)
test['chane_contract'] = test['start date'].map(lambda x: 0)


for i in range(len(date['CL期货连续合约换约日期'])):
    index = test[test['start date'] == date.iloc[i][0]].index.tolist()
    if(len(index) > 0):
        print(index)
        test.iloc[index[0]+1,6] = 1

[15]
[36]
[56]
[79]
[101]
[122]
[144]
[166]
[188]
[210]
[230]
[252]
[276]
[297]
[317]
[340]
[361]
[383]
[405]
[426]
[449]
[471]
[492]
[513]
[536]
[558]
[579]
[601]
[621]
[645]
[666]
[688]
[711]
[731]
[754]
[774]
[798]
[820]
[840]
[861]
[883]
[907]
[926]
[949]
[971]
[993]
[1014]
[1035]
[1059]
[1082]
[1102]
[1123]
[1145]
[1167]
[1188]
[1211]
[1231]
[1254]
[1275]
[1296]
[1319]
[1341]
[1361]
[1383]
[1405]
[1426]
[1448]
[1471]
[1492]
[1514]
[1535]
[1556]
[1580]
[1601]
[1622]
[1645]
[1666]
[1688]
[1710]
[1731]
[1754]
[1776]
[1797]
[1818]
[1841]
[1863]
[1883]
[1906]
[1926]
[1949]
[1971]
[1992]
[2015]
[2036]
[2058]
[2080]
[2103]
[2124]
[2145]
[2167]
[2189]
[2211]
[2232]
[2254]
[2277]
[2297]
[2320]
[2341]
[2362]
[2386]
[2406]
[2426]
[2448]
[2471]
[2491]
[2514]
[2536]
[2557]
[2579]
[2600]
[2623]
[2646]
[2666]
[2687]
[2710]
[2731]
[2753]
[2776]
[2797]
[2819]
[2839]
[2861]
[2885]
[2906]
[2926]
[2949]
[2971]
[2992]
[3014]
[3036]
[3058]
[3080]
[3100]
[3121]
[3146]
[3167]
[3187]
[3210]
[3231]
[3253]


In [184]:
test.to_csv('test.csv')

In [171]:
date.iloc[414][0]

Timestamp('2017-11-20 00:00:00')

In [173]:
test.iloc[18][0]

datetime.datetime(1990, 1, 25, 0, 0)

In [182]:
test.reset_index(drop=True)

,start date,1990-01-01 00:00:00,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,chane_contract
0,1990-01-01 00:00:00,21.65,21.91,21.59,21.82,23615,0
1,1990-01-02 00:00:00,21.8,22.92,21.79,22.89,719,0
2,1990-01-03 00:00:00,23.2,23.8,23,23.68,1566,0
3,1990-01-04 00:00:00,23.88,23.92,22.83,23.41,2500,0
4,1990-01-05 00:00:00,23.25,23.7,23.03,23.08,50,0
5,1990-01-08 00:00:00,22.48,22.6,21.55,21.62,38007,0
6,1990-01-09 00:00:00,21.45,22.12,21.48,22.07,1547,0
7,1990-01-10 00:00:00,22.48,22.92,22.28,22.9,330,0
8,1990-01-11 00:00:00,22.78,23.18,22.52,23.14,858,0
9,1990-01-12 00:00:00,23.2,23.44,23.04,23.13,789,0
